<a href="https://colab.research.google.com/github/Hataeju/Practice/blob/main/part3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. 데이터 준비

### 기본 라이브러리 Import

In [2]:
# 기본라이브러리 import
from google.colab import drive
import os, json, pickle
import subprocess
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

### 라이브러리 추가 설치 및 Import

In [3]:
!pip install pefile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 2.2 MB/s eta 0:00:00


In [4]:
!pip install yara-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.2 MB/s eta 0:00:00


In [5]:
!pip install peid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.9/554.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 10.9 MB/s eta 0:00:00


In [6]:
# 추가 라이브러리 import
import pefile
import yara

### 데이터 로드

In [22]:
# 구글 드라이브 mount
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
# 데이터 파일 위치
colab_path = "gdrive/My Drive/AI보안교육/part3/"


In [9]:
# ! 주석을 해제하여 한번만 실행하세요!
# ! 파일 취급에 주의해주세요!
# ! 명령 실행 후 바로 다시 주석처리 해주세요 !
#!unzip -P 'infected' -qq '/content/gdrive/My Drive/AI보안교육/part3/data_orig/train.zip' -d '/content/gdrive/My Drive/AI보안교육/part3/data_train'
#!unzip -P 'infected' -qq '/content/gdrive/My Drive/AI보안교육/part3/data_orig/test.zip' -d '/content/gdrive/My Drive/AI보안교육/part3/data_test'

In [ ]:
# data_train, data_test에 파일이 압축해제 되어 있는지 확인해 주세요.
# local PC로 다운받지 마세요.
# virus total등 외부 분석 서비스에 업로드 하시면 안됩니다.
# 해당 실습을 위해서만 활용가능합니다. 다른 곳에 공유/사용되면 안됩니다.

In [10]:
# 레이블 파일 로드 => 학습데이터의 초기 Dataframe으로 사용한다.
df_train = pd.read_csv(colab_path + "data_orig/trainSet_label_1000.csv",index_col='hash')
#df_train = df_train[:20] # 처음에는 20개정도의 파일로 Feature Engineering이 잘 되는지 검증 후, 주석 처리하여 전체 파일로 확대한다.

In [11]:
df_train

,index_orig,target
hash,,
ac282b5c526acab44530c0a995d4e23a.vir,6710,1
7a44a4bf7c9a7f332901e6a792e9c8b5.vir,4725,0
1581f524dec2cc3ba8bef37f3ab3d1fd.vir,872,0
15166c91b32b49734932ccae17426f16.vir,862,1
cba8e5374081db7da8d45fa98359d87f.vir,7976,1
...,...,...
e638f5c2fc248ef94a75bbca6923cb57.vir,9004,0
0a9dea1605ae2f00f5e3ab2b41b4548d.vir,419,1
f3bf38b27d8a472be11cb2967fcea3ff.vir,9527,0


In [12]:
# prompt: df_train에 있는 파일명(hash 컬럼)을 가져와서 colab_path 뒤에 data_train 폴더에서 해당 파일을 찾아서 실제로 md5를 구한 다음, 파일명에 있는 md5와 일치하는지 검증하고, 일치하지 않는 것은 출력, 그 후 요약해줘. 비교할때는 파일명에서 앞에 32자만 가지고 비교해

import hashlib

mismatched_files = []
for index, row in df_train.iterrows():
  filename = index
  filepath = os.path.join(colab_path, "data_train", filename)
  try:
    with open(filepath, "rb") as f:
      file_hash = hashlib.md5(f.read()).hexdigest()
    if file_hash[:32] != filename[:32]:
      print(f"Mismatch found: File '{filename}', Calculated MD5: {file_hash}, Filename MD5: {filename[:32]}")
      mismatched_files.append(filename)
  except FileNotFoundError:
    print(f"File not found: {filepath}")
    mismatched_files.append(filename)

print("\n--- Summary ---")
if mismatched_files:
  print(f"Number of files with MD5 mismatch or not found: {len(mismatched_files)}")
  print("List of mismatched or not found files:")
  for f in mismatched_files:
    print(f)
else:
  print("All files matched their filename MD5 or were not found.")



--- Summary ---
All files matched their filename MD5 or were not found.


In [16]:
# data_train폴더의 파일중 PE가 아닌 파일을 출력해줘

for index, row in df_train.iterrows():
  filename = index
  filepath = os.path.join(colab_path, "data_train", filename)
  try:
    pefile.PE(filepath, data=None, fast_load=False)
  except pefile.PEFormatError:
    print(f"{filename} is not a PE file")

512f42a64e7d3c39c1bb2d83b3045193.vir is not a PE file
1bb3f80cd53d1ff4726e3162de6e6db5.vir is not a PE file
ae58c54ba4fe1def507a9863801e5e4a.vir is not a PE file
df5ab3ebf812970ab34accfb030e998c.vir is not a PE file
13ad65eb8ef98915c46681135f5f479d.vir is not a PE file
bbdfc6c0e78c798395274db31a0b5923.vir is not a PE file
3feba3fd7b6a08db84b14a20eb6319dc.vir is not a PE file


KeyboardInterrupt: 

### Feature Engineering


In [17]:
# Part2에서 추출했던 Feature들 실제파일에 적용
# Test데이터에도 동일하게 적용하기 위해 (동일한 Pipeline 구축을 위해) 함수화
def feature_extract(df, mode='train'):
    df['is_PE'] = True
    df['has_error'] = False
    df['size'] = np.nan
    df['packer'] = None
    df['ifc'] = None
    df['section_cnt'] = np.nan
    df['ts_entropy'] = np.nan
    df['rs_size'] = np.nan
    df['yara_https'] = None
    df['file_deleted_cnt'] = np.nan
    df['registry_runkey_modify'] = None

    rule = yara.compile(source='rule https {strings: $a = /https:\S+/ condition: $a}') #https
    for idx, hash in enumerate(df.index.values):
        if idx % 50 == 0:  #전처리 progress를 확인하기 위해
            print(idx)
        if mode == 'train':
            filename = colab_path + 'data_train/' + hash
        else:
            filename = colab_path + 'data_test/' + hash
        try:
            pe = pefile.PE(filename)
            # size
            df.loc[hash, 'size'] = int(os.path.getsize(filename))
            # 파일명은 hash값과 동일하므로 생략한다.
            # 파일엔트로피는 섹션 엔트로피로 대체한다.
            # 패커 여부 탐지 (PEiD 라이브러리 사용)
            packer = subprocess.check_output(['peid', filename]).decode('utf8')
            df.loc[hash, 'packer'] = packer
            # Image File Characteristics
            df.loc[hash, 'ifc'] = hex(pe.FILE_HEADER.Characteristics)
            # 섹션 개수
            df.loc[hash, 'section_cnt'] = int(len(pe.sections))
            # 섹션 엔트로피 & size
            for section in pe.sections:
                if (section.Name).decode('utf8').rstrip('\x00') == '.text':
                    df.loc[hash, 'ts_entropy'] = section.get_entropy()
                if (section.Name).decode('utf8').rstrip('\x00') == '.rsrc':
                    #df.loc[hash, 'rs_size'] = section.SizeOfRawData  #파일 상에서의 section이 차지하는 크기
                    df.loc[hash, 'rs_size'] = section.Misc_VirtualSize #메모리 상에서 차지하는 크기
            # yara rule
            with open(filename, 'rb') as f:
                matches = rule.match(data=f.read())
                if len(matches) != 0:
                    df.loc[hash, 'yara_https'] = True
                else:
                    df.loc[hash, 'yara_https'] = False
            # VT 동적분석 정보 활용
            load_path = colab_path + "vt_json/" + hash.split('.')[0] + ".json"  #.vir 을 제외한 파일명 + .json
            if os.path.exists(load_path):
                with open(load_path) as f:
                    vt_json = json.load(f)
                valid_vendor_cnt = 0
                file_deleted_cnt_total = 0
                if vt_json['meta']['count'] != 0:
                    df.loc[hash, 'file_deleted_cnt'] = 0
                    df.loc[hash, 'registry_runkey_modify'] = False
                for vendor_idx in range(vt_json['meta']['count']):
                     for (key, value) in (vt_json['data'][vendor_idx]['attributes'].items()):
                        if key in ['files_deleted']:
                            file_deleted_cnt_total += len(value)
                            valid_vendor_cnt += 1
                        if key in ['registry_keys_set']:
                            for registry in vt_json['data'][vendor_idx]['attributes']['registry_keys_set']:
                                registry_seperated = registry['key'].lower().split('\\')
                                if (('run' in registry_seperated) or ('runonce' in registry_seperated) or ('runservice' in registry_seperated)):
                                    df.loc[hash, 'registry_runkey_modify'] = True
                if valid_vendor_cnt != 0:
                    df.loc[hash, 'file_deleted_cnt'] = file_deleted_cnt_total/valid_vendor_cnt
        except pefile.PEFormatError:
            df.loc[hash, 'is_PE'] = False
        except Exception as e:
            print(hash)
            print(e)
            df.loc[hash, 'has_error'] = True
    return df


In [18]:
%%time
df_train = feature_extract(df_train, 'train')

0
23c7d5b71318951af6c4588414180225.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/23c7d5b71318951af6c4588414180225.vir']' returned non-zero exit status 1.
2886842048aa3277ee3e3dcb8995d2e2.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/2886842048aa3277ee3e3dcb8995d2e2.vir']' returned non-zero exit status 1.
2e33c858b9063d3df5bf310b0d2b3af7.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/2e33c858b9063d3df5bf310b0d2b3af7.vir']' returned non-zero exit status 1.
8234e8f6d153e84211518f8067ae30e5.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/8234e8f6d153e84211518f8067ae30e5.vir']' returned non-zero exit status 1.
1e7c0b7424adb6cbe86d05df255e78b8.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/1e7c0b7424adb6cbe86d05df255e78b8.vir']' returned non-zero exit status 1.
d7181b31ca65d96629dcf7018b01aee1.vir
Command '['peid', 'gdrive/My Drive/AI보안교육/part3/data_train/d7181b31ca65d96629dcf7018b01aee1.vir']' returned non-zero

In [19]:
df_train

,index_orig,target,is_PE,has_error,size,packer,ifc,section_cnt,ts_entropy,rs_size,yara_https,file_deleted_cnt,registry_runkey_modify
hash,,,,,,,,,,,,,
ac282b5c526acab44530c0a995d4e23a.vir,6710,1,True,False,177189.0,Microsoft Visual Basic\n,0x10f,3.0,5.972551,1504.0,False,NaN,None
7a44a4bf7c9a7f332901e6a792e9c8b5.vir,4725,0,True,False,10244.0,Microsoft Visual C++\nVisual C++ EXE\n,0x10f,3.0,5.609966,NaN,False,0.0,False
1581f524dec2cc3ba8bef37f3ab3d1fd.vir,872,0,True,False,720900.0,MSLRH\nUPX\nUPX (Delphi) Stub\n,0x818f,3.0,NaN,45126.0,False,0.0,False
15166c91b32b49734932ccae17426f16.vir,862,1,True,False,1350176.0,Petite sign ASL for DLL - *old packer \n,0x10f,3.0,NaN,81920.0,False,0.0,False
cba8e5374081db7da8d45fa98359d87f.vir,7976,1,True,False,4035529.0,PECompact\nPECompact\nPeCompact DLL\nPECompact...,0x10f,2.0,7.972003,8192.0,True,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
e638f5c2fc248ef94a75bbca6923cb57.vir,9004,0,True,False,1866612.0,VMProtect\nASProtect\nASProtect\nASProtect\n,0x103,7.0,NaN,5492736.0,True,0.0,False
0a9dea1605ae2f00f5e3ab2b41b4548d.vir,419,1,True,False,513288.0,Petite sign ASL for DLL - *old packer \n,0x10f,3.0,NaN,81920.0,False,0.0,False
f3bf38b27d8a472be11cb2967fcea3ff.vir,9527,0,True,True,140776.0,None,None,NaN,NaN,NaN,None,NaN,None


### 학습용 데이터 저장

In [24]:
# 매 분석시마다 Feature를 추출하는데 시간이 소요되지 않도록 결과 저장
# 전처리/모델링에서 로드하여 사용
df_train.to_csv(colab_path + '/data_with_feature_csv/train_1st.csv')

# 3-1. 분석용 데이터 준비: 전처리

In [26]:
# 저장한 데이터 파일 로드
df_train = pd.read_csv(colab_path + "data_with_feature_csv/train_1st.csv", index_col='hash')

In [27]:
df_train

,index_orig,target,is_PE,has_error,size,packer,ifc,section_cnt,ts_entropy,rs_size,yara_https,file_deleted_cnt,registry_runkey_modify
hash,,,,,,,,,,,,,
ac282b5c526acab44530c0a995d4e23a.vir,6710,1,True,False,177189.0,Microsoft Visual Basic\n,0x10f,3.0,5.972551,1504.0,False,NaN,NaN
7a44a4bf7c9a7f332901e6a792e9c8b5.vir,4725,0,True,False,10244.0,Microsoft Visual C++\nVisual C++ EXE\n,0x10f,3.0,5.609966,NaN,False,0.0,False
1581f524dec2cc3ba8bef37f3ab3d1fd.vir,872,0,True,False,720900.0,MSLRH\nUPX\nUPX (Delphi) Stub\n,0x818f,3.0,NaN,45126.0,False,0.0,False
15166c91b32b49734932ccae17426f16.vir,862,1,True,False,1350176.0,Petite sign ASL for DLL - *old packer \n,0x10f,3.0,NaN,81920.0,False,0.0,False
cba8e5374081db7da8d45fa98359d87f.vir,7976,1,True,False,4035529.0,PECompact\nPECompact\nPeCompact DLL\nPECompact...,0x10f,2.0,7.972003,8192.0,True,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
e638f5c2fc248ef94a75bbca6923cb57.vir,9004,0,True,False,1866612.0,VMProtect\nASProtect\nASProtect\nASProtect\n,0x103,7.0,NaN,5492736.0,True,0.0,False
0a9dea1605ae2f00f5e3ab2b41b4548d.vir,419,1,True,False,513288.0,Petite sign ASL for DLL - *old packer \n,0x10f,3.0,NaN,81920.0,False,0.0,False
f3bf38b27d8a472be11cb2967fcea3ff.vir,9527,0,True,True,140776.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### PE가 아닌 data sample 제외

In [28]:
# PE 포맷의 데이터만 학습에 사용
df_train = df_train[df_train["is_PE"]==True]
len(df_train)

991

### Feature 추출 중 오류가 난 data sample 제외

In [29]:
# Feature 추출에 성공한 데이터만 학습에 사용
df_train = df_train[df_train["has_error"]==False]
len(df_train)

891

### 결측치 처리

In [ ]:
# 결측치 비율 확인
df_train.isnull().sum()/len(df_train)*100

In [ ]:
# 결측치 확인 - packer
df_train[df_train['packer'].isnull()]
# => 알려지지 않은 packer?

In [ ]:
# 결측치 확인 - packer
# 정상/악성 파일별 분포
df_train[df_train['packer'].isnull()].groupby('target').count()
# => 비율이 비슷비슷.'unknown' 으로 대체
# ==> 추후 ('good', 'bad' 'unknown') 으로 인코딩 후 원핫 인코딩 수행

In [ ]:
# 결측치 처리 - packer
df_train['packer'] = df_train['packer'].fillna('unknown')

In [ ]:
# 결측치 확인 - ts_entropy
df_train[df_train['ts_entropy'].isnull()]
# => .text 섹션 자체가 패킹되어 rename?

In [ ]:
# 결측치 처리(대체) - ts_entropy: max 값으로 대체
replace_value_ts_entropy = df_train['ts_entropy'].max().round(2)
df_train['ts_entropy'] = df_train['ts_entropy'].fillna(replace_value_ts_entropy)

In [ ]:
# 결측치 확인 - rs_size
df_train[df_train['rs_size'].isnull()]
# => .rsrc 섹션 자체가 없는 경우 대다수

In [ ]:
# 결측치 처리(대체) - rs_size: 0으로 대체
df_train['rs_size'] = df_train['rs_size'].fillna(0)

In [ ]:
# 결측치 확인 - file_deleted
df_train['file_deleted_cnt'].value_counts(dropna=False)

In [ ]:
# 결측치 처리 - file_deleted: 카테고리화 => 추후 원핫 인코딩
df_train.loc[(df_train['file_deleted_cnt'] > 0), 'file_deleted_cnt']="many"
df_train.loc[(df_train['file_deleted_cnt'] == 0), 'file_deleted_cnt']="zero"
df_train.loc[df_train['file_deleted_cnt'].isnull(), 'file_deleted_cnt']="unknown"

In [ ]:
# 결측치 확인 - registry_runkey_modify
df_train['registry_runkey_modify'].value_counts(dropna=False)

In [ ]:
# 결측치 처리 - registry_runkey_modify: 카테고리화 => 추후 원핫 인코딩
df_train.loc[(df_train['registry_runkey_modify'] == True), 'registry_runkey_modify']="true"
df_train.loc[(df_train['registry_runkey_modify'] == False), 'registry_runkey_modify']="false"
df_train.loc[df_train['registry_runkey_modify'].isnull(), 'registry_runkey_modify']="unknown"

In [ ]:
df_train

In [ ]:
# 결측치 비율 확인
df_train.isnull().sum()/len(df_train)*100

### 이상값 처리

In [ ]:
# BoxPlot으로 확인
# 우리가 직접 데이터를 가공하였으므로, 데이터 생성/수집상의 오류가 있었을 가능성이 낮으므로 모든 값을 학습에 사용
plt.figure(figsize=(16,5))
plt.subplot(1, 4, 1) # 1행 4열의 4개의 그래프 중 첫번째로 state 이동
plt.boxplot([df_train['size']])
plt.xticks([1],['size'])
plt.subplot(1, 4, 2)
plt.boxplot([df_train['section_cnt']])
plt.xticks([1],['section_cnt'])
plt.subplot(1, 4, 3)
plt.boxplot([df_train['ts_entropy']])
plt.xticks([1],['ts_entropy'])
plt.subplot(1, 4, 4)
plt.boxplot([df_train['rs_size']])
plt.xticks([1],['rs_size'])

In [ ]:
# section_cnt 이상치 원본 데이터 확인
df_train[df_train['section_cnt']>80]

### 데이터 정규화

In [ ]:
# 다음 컬럼에 대해 z-score 정규화
columns_to_scale = ['size', 'section_cnt', 'ts_entropy', 'rs_size']
scaler_train = preprocessing.StandardScaler()
scaler_train = scaler_train.fit(df_train[columns_to_scale])
df_train[columns_to_scale] = scaler_train.transform(df_train[columns_to_scale])
df_train

### 데이터 인코딩

In [ ]:
# 범주형 데이터 확인 : packer
df_train['packer'].value_counts(dropna=False)

In [ ]:
# 범주형 데이터 확인 : packer(정상샘플)
packer_list = [val.split('\n') for val in df_train[df_train['target']==0]['packer'].values] #각 value를 줄바꿈 문자열 기준으로 split
packer_list = list(itertools.chain.from_iterable(packer_list)) # 이중 리스트 faltten
normal_packer_set = set(packer_list)
normal_packer_set

In [ ]:
# 범주형 데이터 확인 : packer(악성샘플)
packer_list = [val.split('\n') for val in df_train[df_train['target']==1]['packer'].values] #각 value를 줄바꿈 문자열 기준으로 split
packer_list = list(itertools.chain.from_iterable(packer_list)) # 이중 리스트 faltten
mal_packer_set = set(packer_list)
mal_packer_set

In [ ]:
# 악성샘플과 정상샘플 모두가 사용하는 packer
common_packer = normal_packer_set.intersection(mal_packer_set)
common_packer

In [ ]:
# 범주형 데이터 확인 : image file characteristics
df_train['ifc'].value_counts(dropna=False)[:10]

In [ ]:
# 데이터 원핫 인코딩
df_train = pd.get_dummies(df_train, columns = ['file_deleted_cnt', 'registry_runkey_modify'])

In [ ]:
df_train

# 3-3. 탐색적 분석 : 산포도 분석

In [ ]:
# 히스토그램 - size
hist1 = df_train[df_train['target']==0]['size']
hist2 = df_train[df_train['target']==1]['size']
plt.hist(hist1,label='normal', alpha=0.5, bins=25)
plt.hist(hist2,label='malicious', alpha=0.5, bins=25)
plt.legend(loc=0)

In [ ]:
# 히스토그램 - size (관심영역 초점)
hist1 = df_train[(df_train['target']==0) & (df_train['size']>2)]['size']
hist2 = df_train[(df_train['target']==1) & (df_train['size']>2)]['size']
plt.hist(hist1,label='normal', alpha=0.5, bins=25)
plt.hist(hist2,label='malicious', alpha=0.5, bins=25)
plt.legend(loc=0)

In [ ]:
# 산점도 plot
df_train.plot.scatter(x='section_cnt', y='ts_entropy', c='target', colormap='RdBu_r', alpha=0.5)

### 전처리 프로세스 함수화

In [ ]:
def part3_preprocessing(df, mode='train', replace_ts_entropy=None, scaler=None):
    # PE 포맷의 데이터만 학습에 사용


    # Feature 추출에 성공한 데이터만 학습에 사용


    # 결측치 처리 - packer


    # 결측치 처리(대체) - ts_entropy: max 값으로 대체


    # 결측치 처리(대체) - rs_size: 0으로 대체


    # 결측치 처리 - file_deleted: 카테고리화 => 추후 원핫 인코딩


    # 결측치 처리 - registry_runkey_modify: 카테고리화 => 추후 원핫 인코딩


    # 다음 컬럼에 대해 z-score 정규화
    columns_to_scale = ['size', 'section_cnt', 'ts_entropy', 'rs_size']


    # 데이터 원핫 인코딩


    return df, replace_ts_entropy, scaler

In [ ]:
# 학습 데이터 전처리 (일괄 적용)
df_train = pd.read_csv(colab_path + 'data_with_feature_csv/train_1st.csv', index_col='hash')
df_train, replace_ts_entropy, scaler = part3_preprocessing(df_train, 'train')
df_train

In [ ]:
df_train.isnull().sum()

### 전처리 데이터 저장

In [ ]:
# 추후 모델링을 위해 저장
df_train.to_csv(colab_path + 'data_with_feature_csv/train_processed1.csv')

# 3-4. 모델링 & 3-5. 모델 평가 및 검증

In [ ]:
# 전처리된 파일 로드
df_train = pd.read_csv(colab_path + "data_with_feature_csv/train_processed1.csv", index_col='hash')
df_train

In [ ]:
# 학습할 Feature 선택 및 X,y 분리
target_col = 'target'
cols_to_train = ['size', 'section_cnt', 'ts_entropy', 'rs_size', 'yara_https', 'file_deleted_cnt_many', 'file_deleted_cnt_unknown', 'file_deleted_cnt_zero',
                 'registry_runkey_modify_false', 'registry_runkey_modify_unknown']
#cols_to_train = ['size', 'section_cnt', 'ts_entropy', 'rs_size']
train_y = df_train[target_col]
train_X = df_train[cols_to_train]
train_X

In [ ]:
# 훈련용/평가용 데이터 분할
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.3, random_state=42 )
train_X

In [ ]:
# 모델1 학습 및 평가
model1 = LogisticRegression(C=20, max_iter=1000, random_state=42)
model1.fit(train_X, train_y)
print(model1.score(train_X, train_y))  # accuracy
print(model1.score(val_X, val_y))

In [ ]:
# 모델 2 학습 및 평가
model2 = SVC(kernel = 'linear', probability=True, random_state=42)
model2.fit(train_X, train_y)
print(model2.score(train_X, train_y))  # accuracy
print(model2.score(val_X, val_y))

In [ ]:
# 모델 3 학습 및 평가
model3 = RandomForestClassifier(max_depth = 3, random_state=42)
model3.fit(train_X, train_y)
print(model3.score(train_X, train_y))  # accuracy
print(model3.score(val_X, val_y))

In [ ]:
# 모델 1 저장
save_dict ={
    "model" : model1,
    "target_col": target_col,
    "cols_to_train" : cols_to_train
}
with open(colab_path + 'model/saved_model.pickle','wb') as fw:
    pickle.dump(save_dict, fw)

# 3-6. 모델 적용 및 운영방안 수립

In [ ]:
# 저장된 모델 로드
with open(colab_path + 'model/saved_model.pickle','rb') as f:
    saved_dict = pickle.load(f)
model = saved_dict['model']
target_col = saved_dict['target_col']
cols_to_train = saved_dict['cols_to_train']

In [ ]:
# 테스트용 데이터 로드
df_test = pd.read_csv(colab_path + "data_orig/testSet_label_300.csv", index_col='hash')


In [ ]:
%%time
# feature 추출
df_test = feature_extract(df_test, 'test')

In [ ]:
# 테스트 데이터 전처리 (일괄 적용)
df_test, replace_ts_entropy, scaler = part3_preprocessing(df_test, 'test', replace_ts_entropy, scaler)

In [ ]:
# 전처리된 테스트용 데이터 저장
df_test.to_csv(colab_path + 'data_with_feature_csv/test_processed1.csv')

In [ ]:
# 전처리된 테스트용 데이터 로드
df_test = pd.read_csv(colab_path + 'data_with_feature_csv/test_processed1.csv', index_col='hash')

In [ ]:
# 예측할 Feature 선택 및 X,y 분리
test_y = df_test[target_col]
test_X = df_test[cols_to_train]
test_X

In [ ]:
# 모델 성능 - accuracy
model.score(test_X, test_y)

In [ ]:
# 테스트 데이터에 대한 예측 값 산출
pred_y = model.predict(test_X)

# 모델 성능 - confusion matrix
conf_mat = confusion_matrix(y_true=test_y, y_pred=pred_y)
conf_mat

In [ ]:
# confusion matrix 시각화
plot_confusion_matrix(model, test_X, test_y, normalize=None)

In [ ]:
df_test['target'].value_counts().sort_index()